In [1]:
import numpy as np
import scipy.io
import xarray as xr
from scipy.interpolate import interpn

# Mobley Sea Surface Reflectance Tables
It is important to read the Mobley reference in order to understand the nature of these tables properly.
See the [Surface Reflectance Factor](http://www.oceanopticsbook.info/view/remote_sensing/level_3/surface_reflectance_factors) page at the [Ocean Optics Web Book](http://www.oceanopticsbook.info/).
The tables are radiance ratios, where the numerator is the reflected radiance seen just above water and the denominator is the
sky radiance on the reflected path. If the Mobley surface reflectance factor is denoted $\rho$, then
$$\rho=\frac{L_{wr}}{ L_{sky}},$$
where $L_{sky}$ is the down-welling sky radiance (sky *only*) along the reflected view path and $L_{wr}$ is the *total* up-welling water-reflected radiance 
(including specular sun-glint).
The Mobley tables are provided at a reference wavelength of $\lambda$=550 nm. The wavelength-dependence of $\rho$ can be
quite pronounced if there is a significant amount of sun-glint in the surface-viewing direction.
Proper usage of the reflectance tables therefore also requires some understanding of the spectral issues. Since the reference $L_{sky}$ is blue-biased and the sun-glint (specular) component of reflected radiance is relatively blue-deficient, $\rho$
becomes red-biased in the presence of sun-glint as illustrated in the figure at the [Surface Reflectance Factor](http://www.oceanopticsbook.info/view/remote_sensing/level_3/surface_reflectance_factors) web page.



In [2]:
# Read the sea surface reflectance factors by Mobley
# The header in the file reads as follows :
#  Table of rho values created with FFT surfaces and polarized ray tracing as described in
#    Mobley, C.D., 2015.  Polarized Reflectance and Transmittance Properties of Wind-blown Sea Surfaces,
#    Applied Optics (in review)
#  NOTE: This is a preliminary table for use during review of the above paper.
#    These rho values are for fully developed seas, sun's direct rays parallel to the wind direction,
#    a Rayleigh, single-scattering sky at 550 nm, and wind speeds of 0, 2, 4, 5, 6, 8, 10, 12, 14, and 15 m/s.
#    The final table will be placed online once the above paper has been accepted for publication.
#  rho is computed from the I components of the Stokes radiance vectors: rho = I(surface reflected)/I(sky) [nondimensional]
#  theta_v is the polar viewing angle measured from 0 at the nadir in degrees.
#  _v is the azimuthal viewing angle measured from the sun at phi_v = 0
#  in degrees.
#  Looping order is
#     wind speed
#        sun zenith angle
#           theta_v
#              phi_v
#  Data blocks of 118 theta_v, phi_v, rho(theta_v, phi_v) records on an (f7.1,f9.1,e15.4) format are separated by 
#   records giving the wind speed and sun zenith angle on an (12x,f5.1,20x,f5.1) format.
#-------------------------------
# theta_v    _v       rho
#-------------------------------
#WIND SPEED =  0.0; SUN ZENITH ANGLE =  0.0
mat = scipy.io.loadmat('../Data/WaterReflectance/MobleySeaSurfaceRho2015.mat')

In [3]:
# Data is a 4-dimensional array
# Data table is indexed as follows
mat['SeaSurfRho'][0,0][0].shape

(13L, 10L, 10L, 10L)

In [4]:
# Order of axes is :
# -v (azimuthal viewing angle relative to sun in degrees),
# theta-v (polar viewing angle measured from nadir in degrees)
# sza (solar zenith angle in degrees)
# windspeed (metres per second)
mobley_sea_sur_refl = mat['SeaSurfRho'][0,0][0]
theta_v = np.asarray(mat['SeaSurfRho'][0,0][1], dtype=np.float).squeeze()
phi_v = np.asarray(mat['SeaSurfRho'][0,0][2], dtype=np.float).squeeze()
sza = np.asarray(mat['SeaSurfRho'][0,0][3], dtype=np.float).squeeze()
windspeed = np.asarray(mat['SeaSurfRho'][0,0][4], dtype=np.float).squeeze()
reference = mat['SeaSurfRho'][0,0][5]
axis_definitions = mat['SeaSurfRho'][0,0][6]
axis_order = mat['SeaSurfRho'][0,0][7]
axis_units = mat['SeaSurfRho'][0,0][8]

In [5]:
phi_v

array([   0.,   15.,   30.,   45.,   60.,   75.,   90.,  105.,  120.,
        135.,  150.,  165.,  180.])

In [6]:
theta_v

array([  0. ,  10. ,  20. ,  30. ,  40. ,  50. ,  60. ,  70. ,  80. ,  87.5])

In [7]:
sza

array([  0. ,  10. ,  20. ,  30. ,  40. ,  50. ,  60. ,  70. ,  80. ,  87.5])

In [8]:
windspeed

array([  0.,   2.,   4.,   5.,   6.,   8.,  10.,  12.,  14.,  15.])

In [9]:
reference

array([ u'Mobley, C.D., 2015.  Polarized Reflectance and Transmittance Properties of Wind-blown Sea Surfaces, Applied Optics'], 
      dtype='<U114')

In [10]:
axis_definitions

array([[ array([ u'Theta_v is the polar viewing angle measured from 0 at the nadir in degrees'], 
      dtype='<U74'),
        array([u'Phi_v is the azimuthal viewing angle measured from the sun at phi_v = 0'], 
      dtype='<U71'),
        array([u'Solar Zenith Angle'], 
      dtype='<U18'),
        array([u'Wind Speed'], 
      dtype='<U10')]], dtype=object)

In [11]:
axis_order

array([[array([u'Phi_v'], 
      dtype='<U5'),
        array([u'Theta_v'], 
      dtype='<U7'),
        array([u'SZA'], 
      dtype='<U3'),
        array([u'WindSpeed'], 
      dtype='<U9')]], dtype=object)

In [12]:
axis_units[0][0]

array([u'deg'], 
      dtype='<U3')

In [13]:
# Interpolate at 180 degrees azimuthal viewing angle from sun, _v = 180
# theta_v = 10 - viewing zenith angle from nadir
# sza = 60 - solar zenith angle
# windspeed = 0.5 m/s
reflectance = interpn((phi_v, theta_v, sza, windspeed), mobley_sea_sur_refl, [180.0, 10.0, 60.0, 0.5])

In [14]:
reflectance

array([ 0.02245075])

In [16]:
# Save the data to a .npz file
np.savez('MobleySeaSurfReflTables.npz', phi_v=phi_v, theta_v=theta_v, sza=sza, windspeed=windspeed, 
         mobley_sea_sur_refl=mobley_sea_sur_refl, reference=reference, axis_order=axis_order, 
         axis_definitions=axis_definitions, axis_units=axis_units)